# Using RCAEval benchmark to perform RCA using multi-source telemetry data

In [9]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from RCAEval.utility import download_multi_source_sample
from sklearn.preprocessing import StandardScaler, RobustScaler

from RCAEval.io.time_series import (
    convert_mem_mb,
    drop_constant,
    drop_extra,
    drop_near_constant,
    drop_time,
    preprocess,
    select_useful_cols,
)

from RCAEval.e2e import mmbaro


## Prepare data

1. Download a sample multi-source data
2. Inspect data

In [10]:

# download a sample of multi-source data
download_multi_source_sample()

In [11]:
# read data 
metrics = pd.read_csv("data/multi-source-data/metrics.csv")
logs = pd.read_csv("data/multi-source-data/logs.csv")
traces = pd.read_csv("data/multi-source-data/traces.csv")

# time series transformed from parsed logs
log_ts = pd.read_csv("data/multi-source-data/logts.csv")

# time series transformed from parsed traces latency and response code
trace_ts_lat = pd.read_csv("data/multi-source-data/tracets_lat.csv")
trace_ts_err = pd.read_csv("data/multi-source-data/tracets_err.csv")

with open("data/multi-source-data/inject_time.txt") as ref:
    inject_time = int(ref.read().strip())

In [12]:
# inspect data
metrics.head()

,time,adservice_cpu,cartservice_cpu,checkoutservice_cpu,currencyservice_cpu,emailservice_cpu,frontend_cpu,paymentservice_cpu,productcatalogservice_cpu,recommendationservice_cpu,...,adservice_latency-90,cartservice_latency-90,checkoutservice_latency-90,currencyservice_latency-90,emailservice_latency-90,frontend_latency-90,paymentservice_latency-90,productcatalogservice_latency-90,recommendationservice_latency-90,shippingservice_latency-90
0,1705353846,0.685023,2.216467,0.215886,19.035876,0.279166,6.724489,0.52931,2.636831,2.474853,...,0.004604,0.004993,0.850000,0.110638,0.0046,0.819737,0.008955,0.004418,0.009670,0.004576
1,1705353847,0.685023,2.216467,0.380058,19.035876,0.279166,6.724489,0.52931,2.636858,2.474853,...,0.004594,0.006015,0.899999,0.099482,0.0046,0.823697,0.008955,0.004423,0.009672,0.004576
2,1705353848,0.582707,2.216467,0.380058,19.035876,0.279166,6.724489,0.52931,2.584059,2.474853,...,0.004594,0.006015,0.899999,0.099482,0.0046,0.823697,0.008636,0.004423,0.009672,0.004576
3,1705353849,0.625903,2.216467,0.380058,19.106596,0.279166,6.724489,0.52931,3.250317,2.474853,...,0.004594,0.006015,0.899999,0.099482,0.0046,0.814381,0.008636,0.004416,0.009672,0.004576
4,1705353850,0.613414,2.216467,0.380058,19.106596,0.279166,5.572668,0.52931,3.250317,2.474853,...,0.004594,0.006304,0.899999,0.099482,0.0046,0.814381,0.008636,0.004416,0.009672,0.004576


In [13]:
logs.head()

,time,timestamp,container_name,message,level,req_path,error,cluster_id,log_template
0,21:24,1705353846067349596,currencyservice,conversion request successful,info,NaN,NaN,1.0,currencyservice conversion request successful
1,21:24,1705353846071454981,currencyservice,conversion request successful,info,NaN,NaN,1.0,currencyservice conversion request successful
2,21:24,1705353846073524861,frontend,request started,debug,/product/6E92ZMYYFZ,NaN,2.0,frontend request started
3,21:24,1705353846073599282,frontend,serving product page,debug,/product/6E92ZMYYFZ,NaN,3.0,frontend serving product page
4,21:24,1705353846074646435,currencyservice,conversion request successful,info,NaN,NaN,1.0,currencyservice conversion request successful


In [14]:
traces.head()

,time,traceID,spanID,serviceName,methodName,operationName,startTimeMillis,startTime,duration,statusCode,parentSpanID
0,21:24,46d5d1e1fc25a09db4a81d4d105cee3d,de79080386c4c5c3,currencyservice,Convert,grpc.hipstershop.CurrencyService/Convert,1705353846065,1705353846065999,186,0.0,433c41ba31b519dd
1,21:24,46d5d1e1fc25a09db4a81d4d105cee3d,433c41ba31b519dd,frontendservice,Convert,hipstershop.CurrencyService/Convert,1705353846065,1705353846065431,4133,0.0,6de46372468bc931
2,21:24,46d5d1e1fc25a09db4a81d4d105cee3d,28de2c1a557175ec,frontendservice,Convert,hipstershop.CurrencyService/Convert,1705353846069,1705353846069657,3587,0.0,6de46372468bc931
3,21:24,46d5d1e1fc25a09db4a81d4d105cee3d,a6ccc9fc0e441e5c,currencyservice,Convert,grpc.hipstershop.CurrencyService/Convert,1705353846071,1705353846071000,134,0.0,28de2c1a557175ec
4,21:24,26e74eeaa91cde16a44883aa36a33007,926d999ebf94ce86,frontendservice,NaN,frontend,1705353846073,1705353846073074,222894,NaN,NaN


## Root Cause Analysis 

In [15]:
# preprocess data
metrics = metrics.loc[:, ~metrics.columns.str.endswith("_latency-50")]
metrics = metrics.replace([np.inf, -np.inf], np.nan)

In [19]:
mmdata = {
    "metric": metrics,
    "logs": logs,
    "logts": log_ts,
    "traces": traces,
    "tracets_lat": trace_ts_lat,
    "tracets_err": trace_ts_err,
    "cluster_info": None
}

output = mmbaro(mmdata, inject_time)
ranks = output["ranks"]
print(ranks[:5])

['redis_mem', 'redis_diskio', 'checkoutservice_cpu', 'checkoutservice_mem', 'cartservice_latency-90']
